In [26]:
# Load libraries
import numpy as np

In [27]:
# Set seed
np.random.seed(10)

In [47]:
# Doc params
V = 50
N = 800
K = 3
M = 2

In [48]:
# Set true params
alpha_true = np.array([10, 1, 2])
beta_true = np.random.randint(1, 10, V)

In [49]:
# Generate data
phi_true = np.zeros((K, V))
for k in range(K):
    phi_true[k, :] = np.random.dirichlet(beta_true)

In [50]:
theta_true = np.zeros((M, K))
for m in range(M):
    theta_true[m,:] = np.random.dirichlet(alpha_true)

In [51]:
doc_lens = np.random.randint(100, N, M)
z_true = {}
w = {}
for m in range(M):
    z_true[m] = []
    w[m] = []
    for n in range(doc_lens[m]):
        z_true[m].extend(np.nonzero(np.random.multinomial(1, theta_true[m,:]))[0])
        w[m].extend(np.nonzero(np.random.multinomial(1, phi_true[z_true[m][n], :]))[0])

In [52]:
#z_true = np.zeros((M, N), dtype = "int")
#w = np.zeros((M, N), dtype = "int")
#for m in range(M):
#    for n in range(N):
#        z_true[m, n] = np.nonzero(np.random.multinomial(1, theta_true[m,:]))[0]
#        w[m, n] = np.nonzero(np.random.multinomial(1, phi_true[z_true[m, n], :]))[0]
len(w)

2

In [53]:
# Set initial z randomly
z = {}
for m in range(M):
    z[m] = []
    for n in range(doc_lens[m]):
        z[m].extend(np.nonzero(np.random.multinomial(1, np.ones(K)/K))[0])

In [54]:
# Create count matrices
A = np.zeros((M, K))
for m in range(M):
    for k in range(K):
        A[m, k] = sum(np.array(z[m]) == k)

In [55]:
B = np.zeros((K, V))
for m in range(M):
    for n in range(doc_lens[m]):
        B[z[m][n], w[m][n]] += 1

In [56]:
C = np.zeros(K)
for m in range(M):
    for n in range(doc_lens[m]):
        C[z[m][n]] += 1

In [57]:
# Set initial z randomly
#z = np.zeros((M, N), dtype = "int")
#for m in range(M):
#    for n in range(N):
#        z[m, n] = np.random.binomial(1, 0.5)

In [58]:
# Create count matrices
#A = np.zeros((M, K))
#for m in range(M):
#    for k in range(K):
#        A[m, k] = sum(z[m, :] == k)

In [59]:
#B = np.zeros((K, V))
#for m in range(M):
#    for n in range(N):
#        B[z[m, n], w[m, n]] += 1

In [60]:
#C = np.zeros(K)
#for m in range(M):
#    for n in range(N):
#        C[z[m, n]] += 1

In [61]:
# set hyperparameters alpha and beta
alpha = np.ones(K)
beta = np.ones(V)
#alpha = alpha_true
#beta = beta_true

In [ ]:
# start gibbs sampler
num_iter = 10000
for i in range(num_iter):
    for m in range(M):
        for n in range(doc_lens[m]):
            A[m, z[m][n]] -= 1  # Decrement N1
            B[z[m][n], w[m][n]] -= 1 # Decrement N2
            C[z[m][n]] -= 1 # Decrement N3
            p = np.zeros(K)
            for k in range(K):
                p[k] = (A[m, k] + alpha[k])*((B[k, w[m][n]] + beta[w[m][n]])/(C[k] + sum(beta)))
            p /= sum(p) # This is actually doing k divisions... might be a modest speed up but we can parallelize easily with numba
            z[m][n] = np.nonzero(np.random.multinomial(1, p))[0][0]
            A[m, z[m][n]] += 1 # Increment N1
            B[z[m][n], w[m][n]] += 1 # Increment N2
            C[z[m][n]] += 1 # Increment N3

In [ ]:
# p = np.array([.5, .1, .4])
# np.nonzero(np.random.multinomial(1, p))
?np.nonzero

In [ ]:
# start gibbs sampler
#num_iter = 10000
#for i in range(num_iter):
#    for m in range(M):
#        for n in range(N):
#            A[m, z[m, n]] -= 1
#            B[z[m, n], w[m, n]] -= 1
#            C[z[m, n]] -= 1
#            p = np.zeros(K)
#            for k in range(K):
#                p[k] = (A[m, k] + alpha[k])*((B[k, w[m, n]] + beta[w[m, n]])/(C[k] + sum(beta)))
#            p /= sum(p)
#            z[m, n] = np.nonzero(np.random.multinomial(1, p))[0]
#            A[m, z[m, n]] += 1
#            B[z[m, n], w[m, n]] += 1
#            C[z[m, n]] += 1

In [ ]:
theta = np.zeros((M, K))
for m in range(M):
    for k in range(K):
        theta[m , k] = (A[m, k] + alpha[k])/(doc_lens[m] + sum(alpha))
np.round(theta, 2)

In [ ]:
np.round(theta_true, 2)